In [1]:
import os
import time
import h5py
import torch
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

#kipoi
import kipoi
import kipoi_interpret
from kipoi_veff.utils.plot import seqlogo_heatmap

# Gradient-based methods
from kipoi_interpret.importance_scores.gradient import Gradient, GradientXInput

# utils
from chipseq_utils import *

/home/ubuntu/anaconda3/envs/kipoi-shared__envs__kipoi-py3-keras2/lib/python3.6/site-packages/kipoi_veff/external/concise/seqplotting_deps.py:34: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  min_coords = np.vstack(data.min(0) for data in polygons_data).min(0)
/home/ubuntu/anaconda3/envs/kipoi-shared__envs__kipoi-py3-keras2/lib/python3.6/site-packages/kipoi_veff/external/concise/seqplotting_deps.py:35: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  max_coords = np.vstack(data.max(0) for data in polygons_data).max(0)


In [2]:
### checks
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [29]:
### set the path
# data can be downloaded here: https://github.com/jisraeli/DeepBind
tasks = ['ARID3A', 'CTCFL', 'ELK1', 'FOXA1', 'GABPA', 'MYC', 'REST', 'SP1', 'USF1', 'ZBTB7A']
data_dir = '../../../data/TF-MoDISco/ARID3A_K562_ARID3A_(sc-8821)_Stanford_AC.seq.gz'
#data_dir = '../../../data/TF-MoDISco/CTCFL_K562_CTCFL_(SC-98982)_HudsonAlpha_AC.seq.gz'
#data_dir = '../../../data/TF-MoDISco/ELK1_GM12878_ELK1_(1277-1)_Stanford_AC.seq.gz'
# data_dir = '../../../data/TF-MoDISco/FOXA1_HepG2_FOXA1_(SC-101058)_HudsonAlpha_AC.seq.gz'
# data_dir = '../../../data/TF-MoDISco/GABPA_GM12878_GABP_HudsonAlpha_AC.seq.gz'
# data_dir = '../../../data/TF-MoDISco/MYC_H1-hESC_c-Myc_Stanford_AC.seq.gz'
# data_dir = '../../../data/TF-MoDISco/REST_GM12878_NRSF_HudsonAlpha_AC.seq.gz'
# data_dir = '../../../data/TF-MoDISco/SP1_GM12878_SP1_HudsonAlpha_AC.seq.gz'
# data_dir = '../../../data/TF-MoDISco/USF1_HepG2_USF-1_HudsonAlpha_AC.seq.gz'
# data_dir = '../../../data/TF-MoDISco/ZBTB7A_HepG2_ZBTB7A_(SC-34508)_HudsonAlpha_AC.seq.gz'
result_dir = '../../../out/importance scores/DeepBind/'

### Import model

In [30]:
deep_bind = kipoi.get_model("DeepBind/Homo_sapiens/TF/D00736.002_ChIP-seq_ARID3A")
# deep_bind = kipoi.get_model("DeepBind/Homo_sapiens/TF/D00753.001_ChIP-seq_CTCFL")
# deep_bind = kipoi.get_model("DeepBind/Homo_sapiens/TF/D00360.007_ChIP-seq_ELK1")
# deep_bind = kipoi.get_model("DeepBind/Homo_sapiens/TF/D00761.001_ChIP-seq_FOXA1")
# deep_bind = kipoi.get_model("DeepBind/Homo_sapiens/TF/D00409.004_ChIP-seq_GABPA")
# deep_bind = kipoi.get_model("DeepBind/Homo_sapiens/TF/D00785.001_ChIP-seq_MYC")
# deep_bind = kipoi.get_model("DeepBind/Homo_sapiens/TF/D00799.001_ChIP-seq_REST")
# deep_bind = kipoi.get_model("DeepBind/Homo_sapiens/TF/D00650.007_ChIP-seq_SP1")
# deep_bind = kipoi.get_model("DeepBind/Homo_sapiens/TF/D00700.006_ChIP-seq_USF1")
# deep_bind = kipoi.get_model("DeepBind/Homo_sapiens/TF/D00714.004_ChIP-seq_ZBTB7A")

Using downloaded and verified file: /home/ubuntu/.kipoi/models/DeepBind/downloaded/model_files/Homo_sapiens/TF/D00360.007_ChIP-seq_ELK1/arch/e58e8736e9cf2af6b28c2dce3c90188f
Using downloaded and verified file: /home/ubuntu/.kipoi/models/DeepBind/downloaded/model_files/Homo_sapiens/TF/D00360.007_ChIP-seq_ELK1/weights/75128f40050796bd05bc254635c48593


### Load and process data

In [6]:
chipseq = Chip(data_dir)
train1, valid1, train2, valid2, train3, valid3, alldataset=chipseq.openFile()

../data/TF-MoDISco/ARID3A_K562_ARID3A_(sc-8821)_Stanford_AC.seq.gz


In [28]:
print(len(alldataset))#[0][0].shape
x_data = np.asarray([el[0] for el in alldataset],dtype=np.float32).transpose((0,2,1))
#y_data = np.asarray([el[1] for el in alldataset],dtype=np.float32)
print(x_data.shape)

np.save(result_dir + 'ARID3A_onehot.npy', x_data)

34104
(34104, 101, 4)


### Compute importance scores

In [8]:
grxinp = GradientXInput(deep_bind)
grxinp_scores = grxinp.score(x_data)

In [9]:
print(x_data.shape, grxinp_scores.shape)

(34104, 101, 4) (34104, 101, 4)


In [10]:
np.save(result_dir + 'ARID3A_scores.npy', grxinp_scores)

### Compute hypothetical scores

In [11]:
gr = Gradient(deep_bind)
gr_scores = gr.score(x_data)

In [26]:
means = np.mean(gr_scores, axis=2, keepdims=True)
normalized = gr_scores - means

In [27]:
np.save(result_dir + 'ARID3A_hyp_scores.npy', normalized)